In [1]:
from ipywidgets import Dropdown
import ipywidgets as widgets
from ipywidgets import interact

In [4]:
import altair as alt
import pandas as pd
import geopandas as gpd
import gpdvega

%run theme.py

alt.renderers.enable('notebook')

RendererRegistry.enable('notebook')

In [5]:
data = pd.read_csv("../../datos/procesados/agebs.csv")

In [6]:
geodatos = gpd.read_file('../../datos/procesados/AGEBS_limpios/AGEBS_limpio.shp')

In [7]:
geodatos.set_index("CVEGEO", inplace = True)
data.set_index("CVEGEO", inplace = True)

In [8]:
datos = geodatos.join(data, how = 'right',)

In [9]:
def crea_visualizacion(datos):
    datos.dropna(inplace = True)
    # base
    mapa_base = alt.Chart(datos).mark_geoshape(
        stroke = '#C7C7C7',
        strokeWidth = 0.1,
    ).encode().properties(
        width = 600,
        height = 800,
    )

    # colores
    mapa_resultados = alt.Chart(datos).mark_geoshape(
        fill = 'lightgray',
        stroke = '#C7C7C7',
    ).encode(
        color = alt.Color(
            'Rango de pobreza (%):N',
            legend = alt.Legend(title = "Rango de pobreza",)
        ),
        #tooltip = ['Municipio:N','Enrique Cárdenas Sánchez:Q', 'Alberto Jiménez Merino:Q', 'Miguel Barbosa Huerta:Q', 'Votos nulos:Q', 'No registrados:Q', 'Lista nominal:Q', '% de abstencionismo']
    )
    return mapa_base + mapa_resultados

In [10]:
def filtra_datos(estado, municipio):
    df = datos.copy()
    mask_estado = df['Entidad Federativa'] == estado
    mask_municipio = df['Municipio'] == municipio
    
    working_df = df[mask_estado & mask_municipio].copy()
    
    viz = crea_visualizacion(working_df)
    
    return viz 

In [11]:
estado_widget = Dropdown(options=list(data['Entidad Federativa'].unique()), description = 'Estado')
municipio_widget = Dropdown(options = list(data[data['Entidad Federativa']== estado_widget.value]['Municipio'].unique()), description = 'Muncipio')

In [12]:
def on_state_changed(change):
    municipio_widget.options = list(data[data['Entidad Federativa'] == estado_widget.value]['Municipio'].unique())

In [13]:
estado_widget.observe(on_state_changed, names = 'value')

In [14]:
interact(filtra_datos, estado=estado_widget, municipio = municipio_widget);

interactive(children=(Dropdown(description='Estado', options=('Aguascalientes', 'Baja California', 'Baja Calif…